In [1]:
import math
import collections
import pickle
import random
import scipy
import json

from matplotlib import pyplot as plt

import GPy
import numpy as np
import pandas as pd
from tqdm import tqdm

# from Kernel import MixtureViaSumAndProduct, CategoryOverlapKernel
from InitialData_Gen import initialize
from AcquisitionFunctions import EI, PI, UCB, AcquisitionOnSubspace
# from SamplingCategorical import compute_prob_dist_and_draw_hts
from UpdateCategoricalWeight import compute_reward_for_all_cat_variable, update_weights_for_all_cat_var
from optimization import sample_then_minimize

from AskTell import ask_tell

from scipy.optimize import minimize

from typing import Union, Tuple
from paramz.transformations import Logexp

1. C - List of number of categroies corresponding to each categorical variable
2. Nc - Number of categorical variables
3. Nx - Continuous variables
4. h - categorical
5. x - continuous
6. z - [h, x]

7. nDim - Nc + Nx
8. bounds - Lower and Upper bounds of both Catergorical variables and continuous variables

9. n_iter - Number of iterations to run the algorithm
10. initN - Number of intial data points
11. batch_size (b) - Number of experiments to be generated in each iteration

12. acq_type - Acquisition Type ('EI', 'LCB', 'ThompsonSampling')

### 1. Load Model background from the pickle file

In [ ]:
import sys
myString = sys.path[0]
split_list = myString.split("/")
root = ''
for l in split_list[1:-2]:
    root = root +'/'+ l
    
main_file_path = root + '/RBDJ/'
carbon_source_fiepath = root + '/'

In [2]:
# load data from pkl file
background_file = main_file_path +  "Codes/Round1/1_ModelBackground.pkl"
with open(background_file, "rb") as fp:
    ModelBackground_1 = pickle.load(fp)

### 2. Read Experimental condition and results 

In [3]:
#Load a titer quantification file and create the result array
round0_summary = pd.read_csv(main_file_path +'Exp/Round0/Round0_Result_Summary.csv')
result0 = round0_summary['Specific Productivity'].iloc[:-1,].values.reshape(-1,1)

Exp0 = pd.read_csv(main_file_path +"Codes/Round0/Reconstructed_Round0.csv")
data0 = Exp0.iloc[:,1:].values


In [4]:
#Load a titer quantification file and create the result array
round1_summary = pd.read_csv(main_file_path +'Exp/Round1/Round1_Result_Summary.csv')
result1 = round1_summary['Specific Productivity'].iloc[:-1,].values.reshape(-1,1)

Exp1 = pd.read_csv(main_file_path +"Codes/Round1/Reconstructed_Round1.csv")
data1 = Exp1.iloc[:,1:].values


In [5]:
data = np.concatenate((data0, data1), axis = 0)
result = np.concatenate((result0, result1), axis = 0)


### 3. Update rewards and weights

In [6]:
batch_size = 11
C_list = ModelBackground_1['data_param']['C']

ht_next_list_array = np.atleast_2d(ModelBackground_1['Categorical_dist_param']['ht_batch_list'])

ht_list_rewards = compute_reward_for_all_cat_variable(ht_next_list_array, C_list,
                                                      data, result, batch_size)

Wc_list = update_weights_for_all_cat_var(C_list, 
                ht_list_rewards, ModelBackground_1['Categorical_dist_param']['ht_list'],
                ModelBackground_1['Wc_list'], ModelBackground_1['gamma_list'],
                ModelBackground_1['Categorical_dist_param']['probabilityDistribution_list'],
                batch_size, ModelBackground_1['Categorical_dist_param']['S0'])

0
[-3.92045597 -0.47960078 -1.31372427 -0.28387437 -0.40638091 -7.78541352
 -2.38476325 -2.15687264 -1.63381411 -1.80664713 -0.20264547]


In [7]:
z_next, Categorical_dist_param = ask_tell(data, result, ModelBackground_1['data_param'], 
                                          'RBF', 'thompson_sampling', batch_size, 
                                          Wc_list, ModelBackground_1['gamma_list'])

check2
1
(1000, 1)
(1000, 1, 1)
[[[1.         0.02068382 0.25912445 0.49532358]]]
(1000, 1)
(1000, 1, 1)
[[[2.         0.32133866 0.09901894 0.00461733]]]
(1000, 1)
(1000, 1, 1)
[[[4.         0.00892727 0.03676999 0.39043469]]]
(1000, 1)
(1000, 1, 1)
[[[5.         0.96086313 0.3178407  0.96193733]]]
(1000, 1)
(1000, 1, 1)
[[[6.         0.04526179 0.05977024 0.67905915]]]
(1000, 1)
(1000, 1, 1)
[[[8.         0.02209513 0.2206751  0.46768387]]]
(1000, 1)
(1000, 1, 1)
[[[10.          0.99468886  0.28231993  0.19095397]]]
(1000, 1)
(1000, 1, 1)
[[[1.30000000e+01 3.15385229e-03 8.09358993e-02 1.94140136e-01]]]
(1000, 1)
(1000, 1, 1)
[[[15.          0.01510439  0.9343982   0.36660942]]]
(1000, 1)
(1000, 1, 1)
[[[1.70000000e+01 1.63031654e-02 2.18462812e-01 7.29475170e-02]]]
(1000, 1)
(1000, 1, 1)
[[[1.80000000e+01 8.68476167e-03 4.53224737e-02 1.20643475e-01]]]
       0         1         2         3
0    1.0  0.020684  0.259124  0.495324
1    2.0  0.321339  0.099019  0.004617
2    4.0  0.008

In [8]:
pd.DataFrame(z_next).to_csv('./Round2/2_ExperimentalDesign.csv')



### 4. Store the Model background parameters

In [9]:
ModelBackground_2 = {}
ModelBackground_2 = {'gamma_list': ModelBackground_1['gamma_list'],  'budget': ModelBackground_1['budget'],
                 'bestUpperBoundEstimate': ModelBackground_1['bestUpperBoundEstimate'], 
                     'Wc_list_init': ModelBackground_1['Wc_list_init'],
                   'Wc_list': Wc_list, 'data_param': ModelBackground_1['data_param'], 
                   'Categorical_dist_param': Categorical_dist_param}

import pickle
with open('./Round2/2_ModelBackground.pkl', 'wb') as output:
    # Pickle dictionary using protocol 0.
    pickle.dump(ModelBackground_2, output)

### 5. Conversion to actual experimental execution

In [10]:
Stock_solid = pd.read_excel(carbon_source_fiepath +'CarbonSourceInfo.xlsx', 'Stocks_solid')
Stock_liquid = pd.read_excel(carbon_source_fiepath +'CarbonSourceInfo.xlsx', 'Stocks_liquid')

Carbon_Names = Stock_solid['Carbon Source'].values.tolist()
Carbon_Names.append(Stock_liquid['Carbon Source'][1])
Carbon_Names.append(Stock_liquid['Carbon Source'][2])

Carbon_Ub = [50]*17 #[g/L]
Carbon_Ub.append(10) # Glycerol [mL/L]
Carbon_Ub.append(50) #Ethanol [mL/L]

OG_Gly_Ub = 100 #[mL/L]
Met_Ub = 100 #[mL/L]

Stock_Conc = Stock_solid['g/mL'].values.tolist()
Stock_Conc.append(Stock_liquid['mL/mL'][1].tolist())
Stock_Conc.append(Stock_liquid['mL/mL'][2].tolist())

OG_Stock_Conc = Stock_liquid['mL/mL'][0].tolist()



In [11]:
Selected_Carbon = []
Carbon_mL = []
OG_Gly_mL = []
Met_mL = []


for i in range(batch_size):
    Selected_Carbon.append(Carbon_Names[int(z_next[i,0])])
    temp_factor = Carbon_Ub[int(z_next[i,0])]/ Stock_Conc[int(z_next[i,0])]
    temp = z_next[i,1] * temp_factor * 3
    Carbon_mL.append(temp)
    OG_Gly_mL.append(z_next[i,2] * OG_Gly_Ub * 3/OG_Stock_Conc)
    Met_mL.append(z_next[i,3] * Met_Ub * 3)
    


Experiment_2_3mL = {'Carbon_Type': Selected_Carbon,
               'Conc_Carbon [uL]': Carbon_mL,
               'Gly_OG [uL]': OG_Gly_mL,
               'Met_Prod [uL]': Met_mL}

pd.DataFrame(Experiment_2_3mL).to_csv('./Round2/2_ExperimentPlan_mLValue_3mL.csv')

In [12]:
plt.scatter(data[:,0], data[:,1])
plt.scatter(z_next[:,0],z_next[:,1])


In [13]:
plt.scatter(data[:,2], data[:,3])
plt.scatter(z_next[:,2],z_next[:,3])
